In [ ]:
from datasets import load_from_disk
from pororo import Pororo
from tqdm import tqdm

train_dataset = load_from_disk('../data/train_dataset/train/')
# valid_dataset = load_from_disk('../data/train_dataset/validation/')

ner = Pororo(task="ner", lang="ko")

# ner tags
# PS": "PERSON","LC": "LOCATION","OG": "ORGANIZATION","AF": "ARTIFACT",
# "DT": "DATE","TI": "TIME","CV": "CIVILIZATION","AM": "ANIMAL",
# "PT": "PLANT","QT": "QUANTITY","FD": "STUDY_FIELD",
# "TR": "THEORY","EV": "EVENT","MT": "MATERIAL","TM": "TERM",


In [ ]:
len(train_dataset['context'])

In [ ]:
contexts = train_dataset['context']
print(len(contexts))
contexts = list(set(contexts))
print(len(contexts))

In [ ]:
ner_lists = []
for i in tqdm(range(len(contexts))):
    sample = ner(contexts[i])
    ner_pairs = tuple(set(sample))
#     print(f"pairs len: {len(ner_pairs)}")
    ner_list = []
    for k, v in sample:
        if v != 'O' and k not in ner_list:
            ner_list.append(k)
#     print(f"ner list len: {len(list(set(ner_list)))}")
    ner_lists.append(list(set(ner_list)))
#     print(f"total len: {len(ner_lists)}")
    
print(len(ner_lists))

In [ ]:
len(ner_lists) # 1450 인덱스부터 다시시작

In [ ]:
contexts[1450] = '호박 잎은 심장모양 또는 콩팥모양으로, 커다랗고 거친 털로 덮여 있으며 가장자리가 5개로 얕게 갈라지고 갈래조각에 톱니가 있다 잎자루가 길고, 어긋나기한다. 꽃은 암수한그루로서, 수꽃과 암꽃이 따로 핀다 6월부터 서리가 내릴 때까지 종 모양의 노란 꽃이 잎겨드랑이에 1개씩 달려 핀다 수꽃에만 있는 화분을 벌이 암꽃에 옮기면 수분이 되고, 수분된 암꽃에서 호박이 자란다 암꽃 하나하나가 단 하루만 피어 수분할 수 있는 데다가 호박꽃 대부분이 수꽃이기 때문에 실제로 호박을 생성하는 꽃은 몇 송이밖에 없다 수꽃은 꽃대가 길며 꽃받침통이 얕고 갈래조각의 기부가 꽃부리에 붙어 있으며 암꽃은 꽃대가 짧고 밑부분에 긴 씨방이 있으며 꽃받침갈래조각이 다소 잎같이 된다. 덩굴은 단면이 오각형이고 긴 흰색 솜털이 나있으며 덩굴손으로 감으면서 자라지만 개량된 것은 덩굴성이 아닌 것도 있다. 호박 열매는 여러 종류가 있으며, 모양과 빛깔도 품종에 따라 다르다 원형 또는 타원형으로 껍질이 단단하거나 무르다 속은 결이 거칠고 끈끈한 섬유질로 이루어져 있으며 열매 가운데에 씨가 들어 있다 열매의 무게는 대개 7-14kg 정도이지만 90kg이 넘게 나가는 것도 있다 열매는 대개 오렌지색을 띠며, 노란색·녹색·흰색 등 다른 색깔을 띠는 것도 많다 익으면 바깥면이 주로 짙은 황갈색을 띤다 편평하고 맛이 좋은 씨가 많이 들어 있다 완숙된 종자로 번식한다'

In [ ]:
for i in tqdm(range(len(contexts[1450:]))):
    sample = ner(contexts[i])
    ner_pairs = tuple(set(sample))
#     print(f"pairs len: {len(ner_pairs)}")
    ner_list = []
    for k, v in sample:
        if v != 'O' and k not in ner_list:
            ner_list.append(k)
#     print(f"ner list len: {len(list(set(ner_list)))}")
    ner_lists.append(list(set(ner_list)))
#     print(f"total len: {len(ner_lists)}")

In [ ]:
a = ner_lists
len(a)

In [ ]:
len(ner_lists)

In [ ]:
count = 0
for i in range(len(ner_lists)):
    count += len(ner_lists[i])
print(count)

In [ ]:
ner_lists[0]

In [ ]:
import pandas as pd

def preprocess(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\\n', ' ', text) # remove newline character
    text = re.sub(r'\s+', ' ', text) # remove continuous spaces
    text = re.sub(r'#', ' ', text)

    return text

preprocess_contexts = []
for context in contexts:
    preprocess_contexts.append(preprocess(context))
    
preprocess_ner_lists = []
for i in range(len(ner_lists)):
    preprocess_ner_list = []
    for j in range(len(ner_lists[i])):
        preprocess_ner_list.append(preprocess(ner_lists[i][j]))
    preprocess_ner_lists.append(preprocess_ner_list)

print(len(contexts), len(preprocess_contexts))
print(len(ner_lists), len(preprocess_ner_lists))

longer_preprocess_texts = []
longer_preprocess_titles = []

for i in tqdm(range(len(preprocess_contexts))):
    for j in range(len(preprocess_ner_lists[i])):
        longer_preprocess_texts.append(preprocess_contexts[i])
        longer_preprocess_titles.append(preprocess_ner_lists[i][j])
            
longer_ner_df = pd.DataFrame(data={"text": longer_preprocess_texts, "title": longer_preprocess_titles})
longer_ner_df.to_csv('preprocess_ner_dataset.csv')

longer_ner_df

preprocess_texts = []
preprocess_titles = []

for i in tqdm(range(len(preprocess_contexts))):
    for j in range(len(preprocess_ner_lists[i])):
        if len(preprocess_ner_lists[i][j]) == 1 and not re.sub(r'[가-힣]', '', preprocess_ner_lists[i][j]):
            continue
        else:
            preprocess_texts.append(preprocess_contexts[i])
            preprocess_titles.append(preprocess_ner_lists[i][j])

ner_df = pd.DataFrame(data={"text": preprocess_texts, "title": preprocess_titles})
ner_df.to_csv('preprocess_and_except_one_char_answer_ner_dataset.csv')

print(count, len(preprocess_titles))
# ner_df

In [ ]:
ner_df.describe()

In [ ]:
ner_df

In [ ]:
import pandas as pd
c = pd.read_csv('/home/qa_generation/preprocess_and_except_one_char_answer_ner_dataset.csv')
len(c)

In [ ]:
from pororo import Pororo
from tqdm import tqdm
import os

os.environ["TOKENIZERS_PARALLELISM"] = "true"

qg = Pororo(task="qg", lang="ko")

answers = []
contexts = []
questions = []
for i in tqdm(range(len(c))):
    q = qg(
        c['title'][i],
        c['text'][i]
    )
    answers.append(c['title'][i])
    contexts.append(c['text'][i])
    questions.append(q)

h = pd.DataFrame(data={"context": contexts, "question": questions, "answer": answers})
# h = pd.DataFrame(data={"question": questions, "answer": answers})
h.to_csv('ner_qgzz.csv')

In [ ]:
h

In [ ]:
from pororo import Pororo
from tqdm import tqdm
import os

os.environ["TOKENIZERS_PARALLELISM"] = "true"

qg = Pororo(task="qg", lang="ko")

n = qg(list(c['title']), list(c['text']))
c['question'] = n
c.to_csv('hoow.csv')

In [ ]:
import json

with open("../data/wikipedia_documents.json", "r", encoding="utf-8") as f:
    wiki = json.load(f)
wiki['0']

In [ ]:
wiki['18293']

In [ ]:
train_dataset[0]

In [ ]:
len(wiki)

In [ ]:
# contexts = [wiki[str(i)]['text'] for i in range(len(wiki))]
# contexts = list(set(contexts))
# len(contexts)

In [ ]:
idx = train_dataset['document_id']
len(idx)

In [ ]:
type(wiki)

In [ ]:
0 not in idx

In [ ]:
ner_wiki = {}
for i in range(len(wiki)):
    if i not in idx:
        ner_wiki[str(i)] = wiki[str(i)]
len(ner_wiki)

In [ ]:
ner_wiki['0']['text']

In [ ]:
# pos = Pororo(task="pos", lang="ko")
sts = Pororo(task="similarity", lang="ko")

In [ ]:
# pppp = []
# for k, v in ner(ner_wiki['0']['text']):
#     if v != 'O':
#         pppp.append((k, v))
# list(set(pppp))

ssss = []
for i in range(len(pppp)):
    ssss.append(sts(ner_wiki['0']['text'], pppp[i][0]))
pppp[ssss.index(max(ssss))]



In [ ]:
ner_lists = []
for i in tqdm(range(len(ner_wiki))):
    sample = ner(ner_wiki[str(i)]['text'])
    ner_pairs = tuple(set(sample))
#     print(f"pairs len: {len(ner_pairs)}")
    ner_list = []
    for k, v in sample:
        if v != 'O' and k not in ner_list:
            ner_list.append((k, v))
#     print(f"ner list len: {len(list(set(ner_list)))}")
    ner_lists.append(list(set(ner_list)))
#     print(f"total len: {len(ner_lists)}")